In [1]:
import uproot
import numpy as np
import awkward as ak

import sys
sys.path.append("../..")
from hists.parameters import synchrotronBeamEnergiesMap

In [5]:
tree = uproot.open('/data_cms_upgrade/cuisset/testbeam18/clue3d/v31/cmssw/data/CLUE_clusters.root:clusters')

In [6]:
def findEventsWithLowRecoEnergy(energyFractionThreshold = 0.8, stopAfter=100):
    """ Find events where the total reconstructed energy is less than *energyFractionThreshold* of the beamEnergy (not incl synch losses)
    """
    filtered_arrays = []
    count = 0
    for array in tree.iterate(step_size="100MB", library="ak", cut="beamEnergy==100", 
            filter_name=["event", "ntupleNumber", "beamEnergy", "rechits_energy"]):
        cut = ak.sum(array.rechits_energy, axis=-1)<array.beamEnergy*0.8
        filtered_arrays.append(ak.copy(array[cut]))
        count += np.sum(cut)
        if count > stopAfter:
            break
    return ak.concatenate(filtered_arrays)

In [7]:
findEventsWithLowRecoEnergy()[["ntupleNumber", "event"]]

<Array [{ntupleNumber: 477, ...}, ..., {...}] type='103 * {ntupleNumber: ui...'>

In [8]:
synchrotronBeamEnergiesMap

{20: 20,
 30: 30,
 50: 49.99,
 80: 79.93,
 100: 99.83,
 120: 119.65,
 150: 149.14,
 200: 197.32,
 250: 243.61,
 300: 287.18}

In [18]:
def printForSpreadsheet(ar:ak.Array):
    for event in ar:
        print(";".join([str(int(i)) for i in [event.beamEnergy, event.ntupleNumber, event.event]]))

In [10]:
def findEventsWithNoTrackster():
    pass # TODO

In [11]:
def findEventsWithCLUE3DIssue(thresholdClue3D = 0.8, lowerThresholdRechits=0.9, stopAfter=100):
    filtered_arrays = []
    count = 0
    energyMap = np.vectorize(synchrotronBeamEnergiesMap.get, otypes=[float])
    for array in tree.iterate(step_size="100MB", library="ak", #cut="beamEnergy==100", 
            filter_name=["event", "ntupleNumber", "beamEnergy", "rechits_energy", "clus3D_energy"]):
        if len(array) == 0:
            continue
        synchBeamEnergies = energyMap(array.beamEnergy)

        # fill_none is required to drop events which have no 3D clusters
        cut = ak.fill_none((ak.max(array.clus3D_energy, axis=-1) < thresholdClue3D * synchBeamEnergies) &
         (ak.sum(array.rechits_energy, axis=-1) > lowerThresholdRechits * synchBeamEnergies),
         value=False
        )
        
        cut_array = array[cut]
        filtered_arrays.append(ak.copy(cut_array))
        count += np.sum(cut)
        if count > stopAfter:
            break
    return ak.concatenate(filtered_arrays)

In [12]:
ar = findEventsWithCLUE3DIssue(0.5, stopAfter=1000000000000000)
ar

<Array [{beamEnergy: 300, ...}, ..., {...}] type='547 * {beamEnergy: float3...'>

In [19]:
printForSpreadsheet(ar)

300;435;5155
300;435;8222
20;436;2610
20;436;575
20;436;992
20;436;1326
20;436;1654
20;436;1848
20;436;2427
20;437;5671
20;437;6341
20;437;6411
20;437;7039
20;437;7110
20;437;7298
20;437;10160
20;437;3024
20;437;3154
20;437;3220
20;437;3534
20;437;4238
20;437;8455
20;437;8462
20;437;9772
20;437;780
20;437;793
20;437;1956
20;439;5477
20;439;6059
20;439;6294
20;439;3472
20;439;3579
20;439;4318
20;439;4385
20;439;4525
20;439;4867
20;439;10064
20;439;7494
20;439;8663
20;439;9500
20;439;869
20;439;892
20;441;2812
20;441;4066
20;441;4149
20;441;8062
20;441;8986
20;441;5222
20;441;5726
20;441;5943
20;441;6425
20;441;6575
20;441;9905
20;441;38
20;441;297
20;441;829
20;441;1032
20;441;1703
20;441;1740
20;441;2072
20;442;5376
20;442;5876
20;442;2662
20;442;2828
20;442;10227
20;442;8066
20;442;9449
20;442;225
20;442;722
20;442;1525
20;442;1997
20;442;2245
20;443;1036
20;443;1434
20;443;1439
20;443;1831
20;443;2097
20;444;2484
20;444;3088
20;444;4233
20;444;4249
20;444;4435
20;444;5075
20;444;5265